In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

symbole="ADANIENT"



In [2]:
data=pd.read_csv(f"excel/{symbole}.csv")
#  to remove unnamed column from a excel file
# df = data.loc[:, ~data.columns.str.contains('^Unnamed')]



In [3]:
data.columns

Index(['Date', 'Symbol', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice',
       'High%', 'Low%', 'Close%', 'target_wrt_h&l%', 'stoploss_wrt_h&l%',
       'profit_decider', 'Day', 'Month', 'Year', 'stploss', 'trgt', 'profit',
       'cumsum', 'cumprod'],
      dtype='object')

In [4]:
df=data.groupby(["Year","Month"])

In [5]:
def prfdata1(df1,stploss,trgt):    
    # #  print(df1)
   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])
    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])
    # df1['cumsum']=df1['profit'].cumsum().values
    


    profit = df1['profit'].sum()
    # # #  print(f"Total profit = {profit}% in 2 years")
    # data1 = df1.groupby(['Year', 'Month']).agg({"profit": "sum"})

    # data = 1 + data1.values / 100

    data=df1['profit']/100+1
    # df1['cumprod']=data.cumprod().values
    overallprofit =  data.cumprod()[-1:].values[0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    # #  print(f"Total profit made in 2 years = {formatted_overallprofit}%")

    

    
       
    dic={"tprofit":profit,"cprofit":formatted_overallprofit}
   
    
    return dic


In [6]:
def prfdata(df1,stploss,trgt):    
    # #  print(df1)
   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])
    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])
    df1['cumsum']=df1['profit'].cumsum().values
    


    profit = df1['profit'].sum()
    # # #  print(f"Total profit = {profit}% in 2 years")
    # data1 = df1.groupby(['Year', 'Month']).agg({"profit": "sum"})

    # data = 1 + data1.values / 100

    data=df1['profit']/100+1
    df1['cumprod']=data.cumprod().values
    overallprofit =  data.cumprod()[-1:].values[0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    # #  print(f"Total profit made in 2 years = {formatted_overallprofit}%")

    

    
       
    dic={"tprofit":profit,"cprofit":formatted_overallprofit,"data":df1}
   
    
    return dic


In [7]:
stp = np.arange(0,10,0.01)
trgt =np.arange(0,50,0.1)
def monthaly_processing(df5):
    max=-100000
    fstp=0
    ftrgt=0
    
    for estp in stp:
        print(estp)
        for etrgt in trgt:
            cc=prfdata1(df5,estp,etrgt)        
            if(max<cc['tprofit']):            
                max=cc['tprofit']
                fstp=estp
                ftrgt=etrgt
    return {"fstp":fstp,"ftrgt":ftrgt}
            

        
            


In [8]:
newdf=pd.DataFrame(columns=["symbol","month","year","current_trgt","current_stp","prev_ftrgt","prev_fstp","tp","cp"])

In [9]:
i=0

dhara={}
vivekant=pd.DataFrame()

for partition in df:
    key,partition_data=partition
   

   
    processed_data=monthaly_processing(partition_data)
    
    
    
    if(i==0):
        i=i+1
        Whole_data=prfdata(partition_data,processed_data['fstp'],processed_data["ftrgt"])
        dhara={"prev_stp":processed_data['fstp'],"prev_trgt":processed_data['ftrgt']}
        vivekant=pd.DataFrame(Whole_data['data'])
        vivekant['profit']=vivekant['profit'].apply(lambda x:0)
        vivekant['cumsum']=vivekant['profit'].apply(lambda x:0)
        vivekant['cumprod']=vivekant['profit'].apply(lambda x:0)       
        newdf1=pd.DataFrame({
            "symbol":partition_data['Symbol'][0],
            "month":partition_data['Month'][0],
            "year":partition_data['Year'][0],
            "current_trgt":processed_data['ftrgt'],
            "current_stp":processed_data['fstp'],
            "prev_ftrgt":[0],
            "prev_fstp":[0],
            "tp":[0],
            "cp":[0]})
        #  print("newdf1 1 done")
        
    else:   
        i=i+1
       
        Whole_data=prfdata(partition_data,dhara['prev_stp'],dhara["prev_trgt"])
        #  print("whole done")
        vivekant=pd.concat([vivekant,Whole_data['data']],ignore_index=True)
        #  print("vivekant done")
        tp_cp=prfdata(partition_data,dhara['prev_stp'],dhara['prev_trgt']) 
        #  print("tp_cp done")
        symbol=partition_data['Symbol'].values[0]
        month=partition_data['Month'].values[0]
        year=partition_data['Year'].values[0]     
        #  print("newdf1 no done")
        newdf1=pd.DataFrame({"symbol":symbol,
                             "month":month,
                             "year":year,
                             "current_trgt":[processed_data['ftrgt']],
                             "current_stp":[processed_data['fstp']],
                             "prev_ftrgt":[dhara['prev_trgt']],
                             "prev_fstp":[dhara['prev_stp']],
                             "tp":[tp_cp['tprofit']],
                             "cp":[tp_cp['cprofit']]})
        #  print("newdf1  done")      
        dhara={"prev_stp":processed_data['fstp'],"prev_trgt":processed_data['ftrgt']}        
    newdf=pd.concat([newdf,newdf1],ignore_index=True)
    print(f"{i*100/len(df)}% done!")
    


  
    
    
  

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36
0.37
0.38
0.39
0.4
0.41000000000000003
0.42
0.43
0.44
0.45
0.46
0.47000000000000003
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.96
0.97
0.98
0.99
1.0
1.01
1.02
1.03
1.04
1.05
1.06
1.07
1.08
1.09
1.1
1.11
1.12
1.1300000000000001
1.1400000000000001
1.1500000000000001
1.16
1.17
1.18
1.19
1.2
1.21
1.22
1.23
1.24
1.25
1.26
1.27
1.28
1.29
1.3
1.31
1.32
1.33
1.34
1.35
1.36
1.37
1.3800000000000001
1.3900000000000001
1.4000000000000001
1.41
1.42
1.43
1.44
1.45
1.46
1.47
1.48
1.49
1.5
1.51
1.52
1.53
1.54
1.55
1.56
1.

In [10]:
vivekant.to_csv(f"Using prev month fstp and ftrgt/day_by_day_data/{symbole}.csv",index=False)
newdf.to_csv(f"Using prev month fstp and ftrgt/monthaly_gain_data/{symbole}.csv",index=False)